In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

In [2]:
import gensim
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from gensim.models import KeyedVectors
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import scipy
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import permutation_test_score as p_test

from gensim.models import KeyedVectors

# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format('/Users/yiwen/Documents/coding_practise/blc/mini-dataset/GoogleNews-vectors-negative300.bin', binary=True)
# Access vectors for specific words with a keyed lookup:
vector = model['easy']
# see the shape of the vector (300,)
vector.shape
# Processing sentences is not as simple as with Spacy:
vectors = [model[x] for x in "This is some text I am processing with Spacy".split(' ')]

def open_list(path):
    lst = []
    with open(path,'r') as fin:
        for line in fin:
            line = line.strip()
            lst.append(line)
    return lst

# Three Categories

In [3]:
# three categories
sup_train, basic_train, sub_train = [],[],[]
sup_train_label,basic_train_label,sub_train_label = [],[],[]


sup_test, basic_test, sub_test = [],[],[]
sup_test_label,basic_test_label,sub_test_label = [],[],[]

X_train, X_test, y_train, y_test = [],[],[],[]

path = '/Users/yiwen/Documents/coding_practise/blc/bond/labels.txt'
with open(path,'r') as fin:
    for line in fin:
        line = line.strip()
        line = line.split('\t')
        label,subset,word = line[0],line[1],line[2]
        if label == 'subordinate':
            if subset != 'test':
                sub_train.append(word)
                sub_train_label.append(3)
#                 sub_train_label.append(0)
            else:
                sub_test.append(word)
                sub_test_label.append(3)
#                 sub_test_label.append(0)
        elif label == 'basic':
            if subset != 'test':
                basic_train.append(word)
                basic_train_label.append(2)
            else:
                basic_test.append(word)
                basic_test_label.append(2)
        else:
            if subset != 'test':
                sup_train.append(word)
                sup_train_label.append(1)
#                 sup_train_label.append(0)
            else:
                sup_test.append(word)
                sup_test_label.append(1)
#                 sup_test_label.append(0)
                
print('Training set:',len(sub_train),len(basic_train),len(sup_train))
print('Test set: ', len(sub_test),len(basic_test),len(sup_test))

Training set: 65 58 31
Test set:  40 28 19


# Four categories

In [3]:
# four categories
sup_train, basic_train, sub_train, abst_train = [],[],[],[]
sup_train_label,basic_train_label,sub_train_label,abst_train_label = [],[],[],[]


sup_test, basic_test, sub_test, abst_test = [],[],[],[]
sup_test_label,basic_test_label,sub_test_label,abst_test_label = [],[],[],[]

X_train, X_test, y_train, y_test = [],[],[],[]

# path = '/Users/yiwen/Documents/coding_practise/blc/bond/labels2.txt'
path = '/Users/yiwen/Documents/coding_practise/blc/medium-dataset-train-test-set.txt'
with open(path,'r') as fin:
    for line in fin:
        line = line.strip()
        line = line.split('\t')
        label,subset,word = line[0],line[1],line[2]
        if label == 'subordinate':
            if subset != 'test':
                sub_train.append(word)
#                 sub_train_label.append(3)
                sub_train_label.append(3)
            else:
                sub_test.append(word)
#                 sub_test_label.append(3)
                sub_test_label.append(3)
        elif label == 'basic':
            if subset != 'test':
                basic_train.append(word)
                basic_train_label.append(2)
            else:
                basic_test.append(word)
                basic_test_label.append(2)
        elif label == 'superordinate':
            if subset != 'test':
                sup_train.append(word)
                sup_train_label.append(1)
            else:
                sup_test.append(word)
                sup_test_label.append(1)
        else:
            if subset != 'test':
                abst_train.append(word)
#                 sub_train_label.append(1)
                abst_train_label.append(0)
            else:
                abst_test.append(word)
#                 sup_test_label.append(1)
                abst_test_label.append(0)
                
print('Training set:',len(sub_train),len(basic_train),len(sup_train),len(abst_train))
print('Test set: ', len(sub_test),len(basic_test),len(sup_test),len(abst_test))

Training set: 65 58 31 94
Test set:  40 28 19 98


In [10]:
train_ba,train_sub,train_sup,train_abst = [],[],[],[]
test_ba,test_sub,test_sup,test_abst = [],[],[],[]
dev_ba,dev_sub,dev_sup,dev_abst = [],[],[],[]
with open('/Users/yiwen/Documents/coding_practise/blc/medium-dataset-train-test-set.txt','r') as fin:
    for line in fin:
        line = line.strip()
        line = line.split('\t')
        label,subset,word = line[0],line[1],line[2]
        if label == 'superordinate':
            if subset == 'train':
                train_sup.append(word)
            elif subset == 'dev':
                dev_sup.append(word)
            else:
                test_sup.append(word)
        elif label == 'subordinate':
            if subset == 'train':
                train_sub.append(word)
            elif subset == 'dev':
                dev_sub.append(word)
            else:
                test_sub.append(word)
        elif label == 'basic':
            if subset == 'train':
                train_ba.append(word)
            elif subset == 'dev':
                dev_ba.append(word)
            else:
                test_ba.append(word)   
        else:
            if subset == 'train':
                train_abst.append(word)
            elif subset == 'dev':
                dev_abst.append(word)
            else:
                test_abst.append(word) 
                
                
print('Training_basic:',len(train_ba))
print('Train sub',len(train_sub))
print('Train sup',len(train_sup))
print('Train abst',len(train_abst))
print('Test basic',len(test_ba))
print('Test sub',len(test_sub))
print('Test sup',len(test_sup))
print('Test abst',len(test_abst))
print('Dev basic',len(dev_ba))
print('Dev sub',len(dev_sub))
print('Dev sup',len(dev_sup))
print('Dev abst',len(dev_abst))

Training_basic: 32
Train sub 20
Train sup 14
Train abst 94
Test basic 28
Test sub 40
Test sup 19
Test abst 98
Dev basic 26
Dev sub 45
Dev sup 17
Dev abst 0


In [4]:
# 4-way
mini_synsets = sub_train + basic_train + sup_train + abst_train
print(len(mini_synsets))
test_synsets = sub_test + basic_test + sup_test + abst_test
print(len(test_synsets))

#3-way
# mini_synsets = sub_train + basic_train + sup_train
# print(len(mini_synsets))
# test_synsets = sub_test + basic_test + sup_test
# print(len(test_synsets))


248
185


In [6]:
c = 0 
for x in [sub_train, basic_train, sup_train, abst_train]:
    c += len(x)
    print(len(x))
    
print(c)

c = 0 
for x in [sub_test, basic_test, sup_test, abst_test]:
    c += len(x)
    print(len(x))
    
print(c)

65
58
31
94
248
40
28
19
98
185


In [7]:
#number of feature
word_num = dict()
# with open('/Users/yiwen/Documents/coding_practise/blc/Final_dataset/Buch_40_noun_tagged.csv','r') as fin:
# with open('/Users/yiwen/Documents/coding_practise/blc/mbart/synset_WSD_first_sense_0129_82115.csv','r') as fin:

# DM + BART features
# with open('/Users/yiwen/Documents/coding_practise/blc/full_type_DM/DM_BART_82115.csv','r') as fin:
with open('synset_WSD_first_sense_0129_82115.csv','r') as fin:
    for line in fin:
        line = line.strip()
        line = line.split(';')
        word = line[0]
        features = line[1:]
        word_num[word] = len(features)

#frequency
fre_dict = dict()

with open('/Users/yiwen/Documents/coding_practise/blc/1gramsbyfreq_google_book.txt','r') as fin:
    for line in fin:
        line = line.strip()
        line = line.split('\t')
        word, num = line[0],line[1]
        fre_dict[word] = num
        

#pagerank score
pg_score = dict()

# with open('/Users/yiwen/Documents/coding_practise/blc/mbart/pr_result_82115.csv','r') as fin:
# 
# DM+BART PageRank score
# with open('/Users/yiwen/Documents/coding_practise/blc/full_type_DM/pr_dm_bart_82115.csv','r') as fin:
with open('/Users/yiwen/Documents/coding_practise/blc/mbart/pr_1st_WSD_82115.csv','r') as fin:
    for line in fin:
        line = line.strip()
        line = line.split(',')
        word, pg = line[0], line[3]
        pg_score[word] = pg
        
pg_score['fish.n.01']

'0.002309'

In [8]:
# depth in WN
import nltk
from nltk.corpus import wordnet as wn

syn_depth = dict()

for m in mini_synsets:
    s = wn.synset(m)
    dp = s.max_depth()
    syn_depth[m] = float(dp)
    
    
print(len(syn_depth))

248


In [9]:
#from bottom
syn_hypo = dict()
def GetMaxHeight(synset):
    '''
    returns max height of the tree (from bottom, choosing longest hyponym path)
    '''
    hs = wn.synset(synset).hyponyms()
    if len(hs) < 1:
        return 1.0
    max_height = 1.0
    for h in hs:
        d = GetMaxHeight(h.name()) + 1.0
        if d > max_height:
            max_height = d
    return max_height

# all_synsets = wn.all_synsets('n')
# for a in all_synsets:
#     syn_hypo[m] = GetMaxHeight(a)
    
for m in mini_synsets:
    syn_hypo[m] = GetMaxHeight(m)


#abstractness rating
Ac_rating = dict()

with open('/Users/yiwen/Documents/coding_practise/blc/Abstract_EN_ratings_AoA/AC_ratings_google3m_koeper_SiW.csv','r') as fin:
    heading = True
    for line in fin:
        if heading == True:
            heading = False
            continue
        line = line.strip()
        line = line.split('\t')
        try:
            Ac_rating[line[0]] = line[1]
        except IndexError:
            continue


def bart_feature():
    bart_fea = dict()
#     with open('synset_WSD_0120_82115.csv','r') as fin:
    with open('synset_WSD_first_sense_0129_82115.csv','r') as fin:
#DM BART 
#     with open('/Users/yiwen/Documents/coding_practise/blc/full_type_DM/DM_BART_82115.csv','r') as fin:
            for line in fin:
                line = line.strip()
                line = line.split(';')
                word,features = line[0],line[1:]
                if word in features:
                    features.remove(word)
                bart_fea[word] = len(set(features))
    return bart_fea

bart_result = bart_feature()




def check_child():
    early = dict()
    with open('/Users/yiwen/Documents/coding_practise/blc/mini-dataset/infant_toddler.txt','r') as fin:
        for line in fin:
            line = line.strip()
            line = line.split('\t')
            word,sco = line[0],line[1]
            early[word] = float(sco)/100    
    return early

early_learnt = check_child()

def cue_validity():
    cue_s = dict()
#     with open('cue_validity_82115.csv','r') as fin:
    with open('cue_validity_82115_1st.csv','r') as fin:
#     with open('cue_validity_dm_bart_82115.csv','r') as fin:
        for line in fin:
            line = line.strip()
            line = line.split('\t')
            word,sco = line[0],line[1]           
            cue_s[word] = sco
    return cue_s

bartcue_word = cue_validity()

In [10]:
aoa = dict()
with open('/Users/yiwen/Documents/coding_practise/blc/Abstract_EN_ratings_AoA/synset_first_sense_aoa.csv','r') as fin:
    for line in fin:
        line = line.strip()
        line = line.split('\t')
        aoa[line[0]] = line[1]

# Add AoA

In [90]:
def extracted(word,size,bart,frequency,pagerank,depth,hypo,abrating,child,cue,age,w2v_model):
    threshold = 0.001
    w = word.split('.')[0]
    lst = np.zeros(size)
    for i in range(lst.shape[0]):
        try:#number of features
            lst[0] = float(bart[word])
        except KeyError:
            lst[0] = threshold
        try:# frequency in Google Books
            lst[1] = float(frequency[w])
        except KeyError:
            lst[1] = threshold
        try:
            lst[2] = float(pagerank[word])
        except KeyError:
            lst[2] = threshold
        try:
            lst[3] = depth[word]
        except KeyError:
            lst[3] = threshold
        try:
            lst[4] = hypo[word]
        except KeyError:
            lst[4] = threshold
        try:
            lst[5] = float(abrating[w])
        except KeyError:
#             lst[5] = threshold
            lst[5] = 15
        try:
            lst[6] = child[w]
        except KeyError:
            lst[6] = threshold
        try:
            lst[7] = cue[word]
        except KeyError:
            lst[7] = threshold
        # less characters
        try:
            lst[8] = len(word)
        except KeyError:
            lst[8] = len(word)
        try:
            lst[9] = age[word]
        except KeyError:
            lst[9] = threshold
#         try:
#             lst[10] = edu[word]
#         except KeyError:
#             lst[10] = 10
        try:
            lst[10:] = w2v_model[w]
        except KeyError:
            lst[10:] = np.zeros(lst[10:].shape)
#             lst[10:] = np.ones(lst[10:].shape)
    return lst

In [91]:
dataset = np.zeros((len(mini_synsets),310))
# print(dataset)
for i, m in enumerate(mini_synsets):
    features = extracted(m,dataset.shape[1],bart_result,fre_dict,pg_score,syn_depth,syn_hypo,Ac_rating,early_learnt,bartcue_word,aoa,model)
    dataset[i,:] = features
    if i%50 == 0:
        print('Has processed ',i,' words')
        
print(dataset.shape)

Has processed  0  words
Has processed  50  words
Has processed  100  words
Has processed  150  words
Has processed  200  words
(248, 310)


In [12]:
all_synsets = list(wn.all_synsets('n'))
all_synsets2 = []
for a in all_synsets:
    all_synsets2.append(a.name())

all_synsets2[0]

syn_hypo = dict()
for m in all_synsets2:
    syn_hypo[m] = GetMaxHeight(m)

syn_depth = dict()

for m in all_synsets2:
    s = wn.synset(m)
    dp = s.max_depth()
    
    syn_depth[m] = dp

In [ ]:
import nltk
from nltk.corpus import wordnet as wn

all_synsets = list(wn.all_synsets('n'))
all_synsets[0].name()

with open('synsets_noun20220405.txt','w') as fout:
    for a in all_synsets:
        fout.write(a.name() + '\n')

#dataset_all = np.zeros((len(all_synsets2),309))
dataset_all = np.zeros((len(all_synsets2),310))
# print(dataset)
for i, m in enumerate(all_synsets2):
    features = extracted(m,dataset_all.shape[1],bart_result,fre_dict,pg_score,syn_depth,syn_hypo,Ac_rating,early_learnt,bartcue_word,aoa,model)
    dataset_all[i,:] = features
    if i%10000 == 0:
        print('Has processed ',i,' words')
        
print(dataset_all.shape)

# Train and test (brain-dead version)

In [94]:
save_best_model_output = 999

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from mlxtend.evaluate import permutation_test
import warnings
warnings.filterwarnings("ignore")

w2v = True
log = True
processing_missing_data = True

if w2v:
    dataset2 = dataset
else:
    dataset2 = dataset[:,:10]#number changes

# process missing data, replace missing data with median of all data
# no accuracy improvement
if processing_missing_data:
    for i in range(dataset2.shape[1]):
        median = np.median(dataset2[:,i][dataset2[:,i]!=0])
        dataset2[:,i][dataset2[:,i]==0] = median
    
dataset3 = np.zeros(dataset2.shape)
for i in range(dataset2.shape[1]):
    # take log of frequency and pagerank data
    if log:
        if i in [1,2,7,8]:
            dataset3[:,i] = -np.log(dataset2[:,i])
            dataset3[:,i] = (dataset3[:,i]-np.min(dataset3[:,i]))/(np.max(dataset3[:,i])-np.min(dataset3[:,i]))
            continue
    dataset3[:,i] = (dataset2[:,i]-np.min(dataset2[:,i]))/(np.max(dataset2[:,i])-np.min(dataset2[:,i]))

X = dataset3
y = np.array([3]*len(sub_train)+[2]*len(basic_train)+[1]*len(sup_train) + [0]*len(abst_train))
# print(X.shape,y.shape)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# clf = OneVsRestClassifier(SVC()).fit(X_train, y_train)
# clf = OneVsRestClassifier(SVC(class_weight='balanced')).fit(X, y)

# scores = cross_val_score(clf, X, y, cv=10)*100
# print('accuracy: {:0.2f} ± {:0.2f} %'.format(np.mean(scores), scipy.stats.sem(scores)))



# Change number according to the features
#
# 
dataset_test = np.zeros((len(test_synsets),310))#number changes
# print(dataset)
for i, m in enumerate(test_synsets):
    features = extracted(m,dataset.shape[1],bart_result,fre_dict,pg_score,syn_depth,syn_hypo,Ac_rating,early_learnt,bartcue_word,aoa,model)
    dataset_test[i,:] = features
#     if i%50 == 0:
#         print('Has processed ',i,' words')
# print(dataset_test.shape)

if w2v:
    dataset2 = dataset_test
else:
    dataset2 = dataset_test[:,:10]

# process missing data, replace missing data with median of all data
# no accuracy improvement
if processing_missing_data:
    for i in range(dataset2.shape[1]):
        median = np.median(dataset2[:,i][dataset2[:,i]!=0])
        dataset2[:,i][dataset2[:,i]==0] = median
    

dataset3 = np.zeros(dataset2.shape)
for i in range(dataset2.shape[1]):
    # take log of frequency and pagerank data
    if log:
        if i in [1,2,3,4,6,7,8]:
            dataset3[:,i] = -np.log(dataset2[:,i])
            dataset3[:,i] = (dataset3[:,i]-np.min(dataset3[:,i]))/(np.max(dataset3[:,i])-np.min(dataset3[:,i]+1e-10))
            continue
    dataset3[:,i] = (dataset2[:,i]-np.min(dataset2[:,i]))/(np.max(dataset2[:,i])-np.min(dataset2[:,i]+1e-10))
    
    
    
y_test = np.array([3]*len(sub_test) + [2]*len(basic_test) + [1]*len(sup_test) + [0]*len(abst_test))#number changes


X_test = dataset3

### check which feature is the most important
### by setting other data to 0
#0:num of features,1: frequency in corpus,2:pagerank
#3:depth in WN,4: from bottom,5:abstractness rating,6:early learn 7:cue validity 8: number of characters 9:word2vec

for data in [X, X_test]:
#      data[:,0] = 0
#      data[:,1] = 0
#      data[:,2] = 0
#      data[:,3] = 0
#      data[:,4] = 0
     data[:,5] = 0
     data[:,6] = 0
#      data[:,7] = 0
#      data[:,8] = 0
#      data[:,9:] = 0
#      data[:,10] = 0
#      data[:,11:] = 0
     continue

In [95]:
# revised codes
clf = OneVsRestClassifier(SVC()).fit(X, y)
y_pred = clf.predict(X_test)
result = classification_report(y_test, y_pred, digits=4)
print(result)

              precision    recall  f1-score   support

           0     0.8972    0.9796    0.9366        98
           1     0.8333    0.2632    0.4000        19
           2     0.5833    0.7500    0.6562        28
           3     0.8889    0.8000    0.8421        40

    accuracy                         0.8324       185
   macro avg     0.8007    0.6982    0.7087       185
weighted avg     0.8413    0.8324    0.8186       185



In [31]:
sgdc = SGDClassifier(max_iter=15000, tol=0.01)
print(sgdc)
 
sgdc.fit(X, y)


y_pred = sgdc.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm) 

result = classification_report(y_test, y_pred, digits=4)
print(result)

SGDClassifier(max_iter=15000, tol=0.01)
[[65  7 26  0]
 [ 0  8 11  0]
 [ 0  9 17  2]
 [ 0  3 30  7]]
              precision    recall  f1-score   support

           0     1.0000    0.6633    0.7975        98
           1     0.2963    0.4211    0.3478        19
           2     0.2024    0.6071    0.3036        28
           3     0.7778    0.1750    0.2857        40

    accuracy                         0.5243       185
   macro avg     0.5691    0.4666    0.4337       185
weighted avg     0.7590    0.5243    0.5659       185



# Combine train and test data

In [16]:
dataset = np.zeros((len(mini_synsets)+len(test_synsets),310))
all_synsets = mini_synsets + test_synsets

for i, m in enumerate(all_synsets):
    features = extracted(m,dataset.shape[1],bart_result,fre_dict,pg_score,syn_depth,syn_hypo,Ac_rating,early_learnt,bartcue_word,aoa,model)
#     print(features)
    dataset[i,:] = features
    if i%50 == 0:
        print('Has processed ',i,' words')
        
print(dataset.shape)

Has processed  0  words
Has processed  50  words
Has processed  100  words
Has processed  150  words
Has processed  200  words
Has processed  250  words
Has processed  300  words
Has processed  350  words
Has processed  400  words
(433, 310)


In [17]:
save_best_model_output = 999
w2v = True
log = True
processing_missing_data = False

if w2v:
    dataset2 = np.zeros(dataset.shape)
    dataset2 = dataset
else:
    dataset2 = np.zeros((dataset.shape[0],10))
    dataset2 = dataset[:,:10]#number changes

# process missing data, replace missing data with median of all data
# no accuracy improvement
# if processing_missing_data:
#     for i in range(dataset2.shape[1]):
#         median = np.median(dataset2[:,i][dataset2[:,i]!=0])
#         dataset2[:,i][dataset2[:,i]==0] = median
 

In [18]:
dataset3 = np.zeros(dataset2.shape)
for i in range(dataset2.shape[1]):
    # take log of frequency and pagerank data
    if log:
        if i in [1,2,3,4,6,7,8]:
            dataset3[:,i] = -np.log(dataset2[:,i])
            dataset3[:,i] = (dataset3[:,i]-np.min(dataset3[:,i]))/(np.max(dataset3[:,i])-np.min(dataset3[:,i]+1e-10))
            continue
    dataset3[:,i] = (dataset2[:,i]-np.min(dataset2[:,i]))/(np.max(dataset2[:,i])-np.min(dataset2[:,i]+1e-10))
    


In [ ]:
np.random.seed(0)

prob = np.random.randint(5, size=4) /4
print(prob)

result = np.where(prob > 0.0000000001, prob, -10)
# print(result)
np.log10(result, out=result, where=result > 0)
print(result)

In [ ]:
X = dataset[:len(mini_synsets)]
y = np.array([3]*len(sub_train)+[2]*len(basic_train)+[1]*len(sup_train) + [0]*len(abst_train))

# 3-way classification
# y = np.array([3]*len(sub_train)+[2]*len(basic_train)+[1]*len(sup_train))

# 4-way classification
y_test = np.array([3]*len(sub_test) + [2]*len(basic_test) + [1]*len(sup_test) + [0]*len(abst_test))#number changes

# 3-way classification
# y_test = np.array([3]*len(sub_test) + [2]*len(basic_test) + [1]*len(sup_test))
X_test = dataset[len(mini_synsets):]

### check which feature is the most important
### by setting other data to 0
#0:num of features,1: frequency in corpus,2:pagerank
#3:depth in WN,4: from bottom,5:abstractness rating,6:early learn 7:cue validity 8: number of characters 9:word2vec

for data in [X, X_test]:
#     data[:,0] = 0
#     data[:,1] = 0
#     data[:,2] = 0
#     data[:,3] = 0
#     data[:,4] = 0
#     data[:,5] = 0
#     data[:,6] = 0
#     data[:,7] = 0
#     data[:,8] = 0
#     data[:,9] = 0
#     data[:,10:] = 0
    continue

# revised codes
# clf = OneVsRestClassifier(SVC()).fit(X, y)
clf = OneVsRestClassifier(SVC(kernel = 'linear')).fit(X, y)
y_pred = clf.predict(X_test)
result = classification_report(y_test, y_pred, digits=4)
print(result)

In [ ]:
print(y_pred)
print(y_test)

In [24]:
# from sklearn.linear_model import SGDClassifier
# clf = SGDClassifier(SVC(kernel = 'linear')).fit(X, y)
# y_pred = clf.predict(X_test)
# result = classification_report(y_test, y_pred, digits=4)
# print(result)

# sgdc = SGDClassifier(max_iter=12000, tol=0.02) 87 70 74.3
sgdc = SGDClassifier(max_iter=15000, tol=0.01)
print(sgdc)
 
sgdc.fit(X, y)


y_pred = sgdc.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm) 

result = classification_report(y_test, y_pred, digits=4)
print(result)

SGDClassifier(max_iter=15000, tol=0.01)
[[18  1  0]
 [ 5 22  1]
 [ 5  3 32]]
              precision    recall  f1-score   support

           1     0.6429    0.9474    0.7660        19
           2     0.8462    0.7857    0.8148        28
           3     0.9697    0.8000    0.8767        40

    accuracy                         0.8276        87
   macro avg     0.8196    0.8444    0.8192        87
weighted avg     0.8586    0.8276    0.8326        87



In [ ]:
from mlxtend.evaluate import permutation_test

In [60]:
permutation_test(y_pred,y_pred2,method='approximate',num_rounds=10000,seed=0)

9.999000099990002e-05

In [24]:
# original codes
clf = OneVsRestClassifier(SVC).fit(X, y)
y_pred = clf.predict(X_test)
result = classification_report(y_test, y_pred, digits=4)
print(result)

0.9665033496650335

In [ ]:
if len(X[X!=0]) == 0:
    X = np.zeros((X.shape[0],1))
else:
    X = X[:,~np.all(X==0, axis=0)]
if len(X_test[X_test!=0]) == 0:
    X_test = np.zeros((X_test.shape[0],1))
else:
    X_test = X_test[:,~np.all(X_test==0, axis=0)]
    
epoch = 1
output_size = y_test.shape[0]
BLC_f1s = np.zeros((epoch,))
p_values = np.zeros((epoch,))
print(X.shape, X_test.shape)
for i in range(save_best_model_output):
    for i in range(epoch):
#         clf = OneVsRestClassifier(SVC(class_weight = 'balanced',random_state = 0)).fit(X, y)
        clf = OneVsRestClassifier(SVC()).fit(X, y)
        sgd_predictions = clf.predict(X_test)
#         sgd_classifier = SGDClassifier(class_weight = 'balanced',max_iter = 10000,n_iter_no_change=200)
#         sgd_classifier.fit(X, y)
#         sgd_predictions = sgd_classifier.predict(X_test)
        result = classification_report(y_test, sgd_predictions, digits=4,output_dict=True)
        BLC_f1s[i] = result['2']['f1-score']

        if save_best_model_output > 1:
            best_perm = sgd_predictions
#             best_perm[best_perm!=2] = 0
        elif best_perm is None:
            pass
        else:
            pred_perm = sgd_predictions
#             pred_perm[pred_perm!=2] = 0
            p_values[i] = permutation_test(pred_perm,best_perm,method='approximate', num_rounds=10000)

    # print('f1-score for BLC is ', BLC_f1s)
    print('f1-score average -> {:.2f} ± {:.2f} %'.format(np.mean(BLC_f1s)*100,scipy.stats.sem(BLC_f1s)*100))
    if best_perm is None:
        print("Pick best model first!")
    elif save_best_model_output == 1:
        print('p value of current model vs. best model -> {:.4f} ± {:.4f} '.format(np.mean(p_values),scipy.stats.sem(p_values)))
        print('individual p value -> ', p_values)
    if np.mean(BLC_f1s)>0.20 and save_best_model_output != 1:
        save_best_model_output = 1
        print('model saved!')
        break

In [ ]:
y_pred = sgd_predictions.tolist()
print(y_pred)

In [ ]:
y_true = y_test.tolist()

# All BLC prediction 

In [ ]:
if w2v:
    dataset_all2 = dataset_all
else:
    dataset_all2 = dataset_all[:,:9]

# process missing data, replace missing data with median of all data
# no accuracy improvement
if processing_missing_data:
    for i in range(dataset_all2.shape[1]):
        median = np.median(dataset_all2[:,i][dataset_all2[:,i]!=0])
        dataset_all2[:,i][dataset_all2[:,i]==0] = median
    
dataset_all3 = np.zeros(dataset_all2.shape)
for i in range(dataset_all2.shape[1]):
    # take log of frequency and pagerank data
    if log:
        if i in [1,2,7]:
            dataset_all3[:,i] = -np.log(dataset_all2[:,i])
            dataset_all3[:,i] = (dataset_all3[:,i]-np.min(dataset_all3[:,i]))/(np.max(dataset_all3[:,i])-np.min(dataset_all3[:,i]))
            continue
    dataset_all3[:,i] = (dataset_all2[:,i]-np.min(dataset_all2[:,i]))/(np.max(dataset_all2[:,i])-np.min(dataset_all2[:,i]))


# result = dict()
# for i in range(dataset_all3.shape[0]):
#     result[all_synsets2[i]] = int(clf.predict(dataset_all3[i:i+1,:7])[0])
# # dataset_all[0].shape
result = sgd_classifier.predict(dataset_all3)

In [ ]:
result

In [ ]:
with open('predicted_sub_02-1_sgd.txt','w') as fout:
    for i in range(len(result)):
        if result[i] == 3:
            fout.write(all_synsets2[i])
            fout.write('\n')

In [ ]:
with open('predicted_BLC_02-1_sgd.txt','w') as fout:
    for i in range(len(result)):
        if result[i] == 2:
            fout.write(all_synsets2[i])
            fout.write('\n')
# dataset_all[0].shape

In [ ]:
predicted_blc = []
with open('predicted_BLC_0131_sgd.txt','r') as fin:
    for line in fin:
        line = line.strip()
        predicted_blc.append(line)

In [ ]:
import random

random.shuffle(predicted_blc)
predicted_blc[:250]

In [ ]:
for p in predicted_blc[:250]:
    print(p)

In [ ]:
predict_word = set()
for p in predicted_blc:
    p = wn.synset(p).lemmas()[0]
    predict_word.add(p.name())
    
print(len(predict_word))

In [ ]:
with open('predict_word_sgd798.txt','w') as fout:
    for p in predict_word:
        fout.write(p)
        fout.write('\n')

In [ ]:
random_250 = set()
with open('predict_word_sgd798.txt','r') as fin:
    for line in fin:
        line = line.strip()
        if len(random_250) == 250:
            break
        else:
            random_250.add(line)

# Cue validity

In [ ]:
bart_result['animal.n.01']

In [ ]:
def bart_pair():
    bart_fea = dict()
#     with open('synset_WSD_0120_82115.csv','r') as fin:
#     with open('synset_WSD_first_sense_0129_82115.csv','r') as fin:
    with open('/Users/yiwen/Documents/coding_practise/blc/full_type_DM/DM_BART_82115.csv','r') as fin:
        for line in fin:
            line = line.strip()
            line = line.split(';')
            word,features = line[0],line[1:]
            if word in features:
                features.remove(word)
            bart_fea[word] = set(features)
    return bart_fea

In [ ]:
bart_pairs = bart_pair()

In [ ]:
bart_pairs['birch.n.01']

In [ ]:
def total_features(dct):
    total = []
    for k,v in dct.items():
        for vi in v:
            total.append(vi)
    return total

# all_features = []
# for k,v in features_buch.items():
#     for vi in v:
#         all_features.append(vi)

bart_features = total_features(bart_pairs)

In [ ]:
bart_features[0]

In [ ]:
def cue_number(total_fea):
    q_num = dict()
    for i in total_fea:
        if i in q_num.keys():
            q_num[i]+=1
        else:
            q_num[i]=0
            q_num[i]+=1
    return q_num

bartcue_num = cue_number(bart_features)

In [ ]:
bartcue_num['cat.n.01']#有33个synsets带有cat.n.01这个feature

In [ ]:
import math
def cue_category(q_num,total_fea,all_words):
    cue_ctgy = dict()
    for i in all_words:
        c = 0
        for f in total_fea[i]:
            mks = 1/q_num[f]
            c+=mks
        cue_ctgy[i] = c
    return cue_ctgy


In [ ]:
hyper = lambda s: s.hypernyms()
list(wn.synset('sea.n.01').closure(hyper))

In [ ]:
wn.synset('thing.n.12').definition()

In [ ]:
def hyper_(syn):
    hp = lambda s: s.hypernyms()
    tmp =  list(wn.synset(syn).closure(hp))
    up = []
    for t in tmp:
        up.append(t.name())
    return up

In [ ]:
categories = ['animal.n.01','plant.n.02','artifact.n.01','natural_phenomenon.n.01','matter.n.03','thing.n.12']
bart_world = set()
for k in bart_pairs.keys():
    up = hyper_(k)
    for c in categories:
        if c in up:
            bart_world.add(k)
            continue

In [ ]:
bartcue_word = cue_category(bartcue_num,bart_pairs,bart_pairs.keys())

In [ ]:
len(bartcue_word)

In [ ]:
# with open('cue_validity_82115_1st.csv','w') as fout:
with open('cue_validity_dm_bart_82115.csv','w') as fout:
    for k,v in bartcue_word.items():
        content = k + '\t' + str(v) + '\n'
        fout.write(content)

In [ ]:
bart_cue_word_sorted = sorted(bartcue_word.items(), key=lambda x: x[1], reverse=True)

In [ ]:
mini_data = []
with open('/Users/yiwen/Documents/coding_practise/blc/Final_dataset/synset_word_40_mapping.txt','r') as fin:
    for line in fin:
        line = line.strip()
        line = line.split('\t')
        syn = line[0]
        mini_data.append(syn)

In [ ]:
sub = mini_data [0:10]
basic = mini_data [11:20]
sup = mini_data[21:30]
abst = mini_data[31:40]

categories = [sub,basic,sup,abst]

In [ ]:
for c in categories:
    avg = 0.0
    for w in c:
        avg += float(bartcue_word[w])
    print(c,avg)

# Mini-world

In [ ]:
bird_list = ['bird','penguin','eagle','emu','falcon','owl','robin','parakeet','partridge','peacock','pelican','gull','seagull','pheasant','pigeon','quill']
fish_list = ['fish','reef','shark','dolphin','whale','rod','salamander','salmon','sardine','walrus','mackerel','pike','tuna','trout']
domestic_list = ['rooster','duck','swan','chicken','pig','cow','ox','sheep','ram','ewe','goat']
sea_animal = ['seafood','turtle','lobster','oyster','octopus','shell','clam','snail','platypus','squid','shrimp']
food_list = ['meat','flesh','snack','cornbeef','beef','fry','veal','mutton','lamb','toast','hamburger','gravy','bouillon','bread','pizza','pepperoni','bologna','pie','pastry','pasta','cake','pancake','noodle']
spice_list = ['spice','garlic','sugar','herb','pepper','salt','parsley','herb','mint','oregano','onion']
horse_list = ['horse','bronco','zebra','donkey','colt']
insect_list = ['insect','hornet','bee','butterfly']
rodent_list = ['rodent','mouse','chipmunk','hamster','porcupine','squirrel']
mammal_list = ['mammal','cat','tiger','lion','leopard','cheetah','cougar','bat_animal','seal','beaver','otter','mink','mongoose','bison','buffalo','bull','camel','deer','caribou','moose','elk','fawn','dog','fox','wolf','coyote','hyena','chimp','monkey','gorilla','chimpanzee','elephant','giraffe','rabbit']
animal_list = ['animal','snake','reptile','lizard','python','rattlesnake','bear','grizzly']
flower_list = ['flower','violet','dandelion','rose','tulip','orchid']
vegetable_list = ['vegetable','zucchini','cucumber','cabbage','carrot','yam','turnip','tomato','spinach','rhubarb','radish','pumpkin','potato','pickle','pea','cauliflower','artichoke','bean']
plant_list = ['plant','tree','seed','oak','birch','cactus','forest','wood','willow','vine','shrub','seaweed','coral','wheat','oat','weed','grass','nut','peanut','walnut','mushroom','fungus','moss']
fruit_list = ['fruit','grapefruit','apple','orange','tangerine','walnut','watermelon','cantaloupe','cherry','strawberry','raspberry','blueberry','raisin','plum','pineapple','pear','peach','olive','mandarin','nectarine','lime','lemon','grape','fig','coconut','banana']
vehicle_list = ['vehicle','submarine','ship','boat','train','bus','subway','helicopter','jeep','van','car','cab','caravan','scooter','jet','plane','truck','bicycle','tricycle','unicycle','motorcycle']
all_list = bird_list + fish_list + insect_list + sea_animal + domestic_list + food_list + spice_list + horse_list + animal_list + mammal_list + rodent_list + plant_list + flower_list +vehicle_list + vegetable_list +fruit_list


In [ ]:
list_names = [bird_list,fish_list,domestic_list,food_list,spice_list,sea_animal,horse_list,animal_list,insect_list,mammal_list,rodent_list,flower_list,vegetable_list,plant_list,fruit_list,vehicle_list,]
len_animals = len(bird_list) + len(fish_list) + len(domestic_list) + len(sea_animal) + len(horse_list) + len(insect_list) + len(rodent_list) + len(mammal_list) + len(animal_list)
print(len_animals,'animals')
len_plants = len(spice_list) + len(flower_list) + len(vegetable_list) + len(plant_list) + len(fruit_list)
print(len_plants,'plants')
print(len(vehicle_list),'vehicles')
print(len(food_list))

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

features_buch = dict()
with open('/Users/yiwen/Documents/coding_practise/blc/Buch_4436_processed.csv','r') as fin:
    for line in fin:
        line = line.strip()
        line = line.split(';')
        word,feature = line[0],line[1]
        word = lemmatizer.lemmatize(word)
        if word == 'bike':
            word = 'bicycle'
            features_buch[word].add(feature)
        else:
            if word in all_list:
                if word in features_buch.keys():
                    features_buch[word].add(feature)
                else:
                    features_buch[word] = set()
                    features_buch[word].add(feature)

In [ ]:
def total_features(dct):
    total = []
    for k,v in dct.items():
        for vi in v:
            total.append(vi)
    return total

# all_features = []
# for k,v in features_buch.items():
#     for vi in v:
#         all_features.append(vi)

all_features = total_features(features_buch)

In [ ]:
def cue_number(total_fea):
    q_num = dict()
    for i in total_fea:
        if i in q_num.keys():
            q_num[i]+=1
        else:
            q_num[i]=0
            q_num[i]+=1
    return q_num

cue_num = cue_number(all_features)
# cue_num = dict()
# for a in all_features:
#     if a in cue_num.keys():
#         cue_num[a]+=1
#     else:
#         cue_num[a] = 0
#         cue_num[a]+=1

def cue_category(q_num,total_fea,all_words):
    cue_ctgy = dict()
    for i in all_words:
        c = 0
        for f in total_fea[i]:
            mks = 1/q_num[f]
            c+=mks
        cue_ctgy[i] = c
    return cue_ctgy

# cue_word = dict()
# for i in all_list:
#     init = 0
#     for f in features_buch[i]:
#         mks = 1/cue_num[f]
#         init += mks
#     cue_word[i] = init

cue_word = cue_category(cue_num,features_buch,all_list)

In [ ]:
category_score = dict()
for n in list_names:
    category_score[n[0]] = dict()
    for i in n:
        print(i,cue_word[i])
        category_score[n[0]][i] = cue_word[i]
    print(len(n),' ',n[0],'____')

In [ ]:
type_list = ['vehicle','transportation','animal','plant','fruit','bird','feather']
features_buch2 = dict()
with open('/Users/yiwen/Documents/coding_practise/blc/Buch_4436_processed.csv','r') as fin:
    for line in fin:
        line = line.strip()
        line = line.split(';')
        word,feature = line[0],line[1]
        if line[2] == 'noun':
            if word in features_buch2.keys():
                features_buch2[word].add(feature)
            else:
                features_buch2[word] = set()
                features_buch2[word].add(feature)

selected_word = set()

for k,v in features_buch2.items():
    for vi in v:
        if vi in type_list:
            selected_word.add(k)

In [ ]:
all_features2 = total_features(features_buch2)
cue_num2 = cue_number(all_features2)
cue_word2 = cue_category(cue_num2,features_buch2,selected_word)